# Next bigger number with the same digits

Create a function that takes a positive integer and returns the next bigger number that can be formed by rearranging its digits. For example:

```
12 ==> 21
513 ==> 531
2017 ==> 2071
```

```
nextBigger(num: 12)   // returns 21
nextBigger(num: 513)  // returns 531
nextBigger(num: 2017) // returns 2071
```

If the digits can't be rearranged to form a bigger number, return `-1`:

```
9 ==> -1
111 ==> -1
531 ==> -1
```

```
nextBigger(num: 9)   // returns nil
nextBigger(num: 111) // returns nil
nextBigger(num: 531) // returns nil
```

Source: [Next bigger number with the same digits](https://www.codewars.com/kata/55983863da40caa2c900004e/train/python)


## Brute force

My first approach was to generate all possible numbers. Then sort them. In the created list find the given number (the `n`) and then collect next one.

First problem: there are duplication in the permutations - added `set` structure for removing the result.

Second problem: there is a case, where there is not next number (some focus on different part of the task).

Finally I created solution which pass the first unit tests:

In [1]:
from itertools import permutations

def next_bigger1(n):
    all_permutations = map(int, map(lambda x: ''.join(x), permutations(str(n))))
    bigger_than_n_no_duplicate = set(filter(lambda x: x > n, all_permutations))
    
    if len(bigger_than_n_no_duplicate) == 0:
        return -1

    return sorted(bigger_than_n_no_duplicate)[0]

assert next_bigger1(12) == 21
assert next_bigger1(513) == 531
assert next_bigger1(2017) == 2071
assert next_bigger1(414) == 441
assert next_bigger1(144) == 414


Unfortunately... this method work much too long for number like `59884848459853`. Even the generators magic didn't help much. I need change approach.

## No permutation are needed

After looking at the problem, I found, that I should checking the digits one by one. Since the second to the end, up to the front. Checking is about verification if on the right of tested digit there is a digit minimal bigger than it. If yes, then swap the places and sort all digits on the right (to get the lowest possible number).

In [2]:
def next_bigger2(n):
    digits = list(map(int, str(n)))
    digits_number = len(digits)
    
    for i in range(digits_number - 2, -1, -1):
        tested_digit = digits[i]
        digits_on_right = digits[i + 1:]
        bigger_than_tests = sorted(filter(lambda d: d > tested_digit, digits_on_right))
        if len(bigger_than_tests) == 0:
            continue

        digit_to_replace = bigger_than_tests[0]
        
        digits_on_right[digits_on_right.index(digit_to_replace)] = tested_digit
        new_digits_orders = digits[:i] + [digit_to_replace] + sorted(digits_on_right)

        return int(''.join(map(str, new_digits_orders)))

    return -1

assert next_bigger2(12) == 21
assert next_bigger2(513) == 531
assert next_bigger2(2017) == 2071
assert next_bigger2(414) == 441
assert next_bigger2(144) == 414
assert next_bigger2(198765432) == 213456789

Which fulfilled the task.

## Improvements

Range seems to replace with enumerator.

In [3]:
def next_bigger3(n):
    digits = list(map(int, str(n)))
    digits_number = len(digits)

    for i, tested_digit in enumerate(reversed(digits)):
        digits_on_right = digits[digits_number - i:]
        bigger_than_tests = sorted(filter(lambda d: d > tested_digit, digits_on_right))
        if len(bigger_than_tests) == 0:
            continue

        digit_to_replace = bigger_than_tests[0]
        
        digits_on_right[digits_on_right.index(digit_to_replace)] = tested_digit
        new_digits_orders = digits[:digits_number - i - 1] + [digit_to_replace] + sorted(digits_on_right)

        return int(''.join(map(str, new_digits_orders)))

    return -1

assert next_bigger3(12) == 21
assert next_bigger3(513) == 531
assert next_bigger3(2017) == 2071
assert next_bigger3(414) == 441
assert next_bigger3(144) == 414
assert next_bigger3(987654321) == -1
assert next_bigger3(198765432) == 213456789